<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/129_Docker_Container_Creation_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 🚀 Docker Project Checklist (1-Page)

### 📂 Project Setup

```bash
mkdir myproject && cd myproject   # create project folder
nano Dockerfile                   # create Dockerfile
nano .dockerignore                 # create .dockerignore
nano requirements.txt              # runtime deps
nano requirements-dev.txt          # dev tooling
nano main.py                       # entry/test script
nano claude_chat.py                # (your Claude helper functions)
nano .env                          # API keys (local only, not copied into image)
```

### 📄 Typical Files

* **Dockerfile** → recipe for environment
* **.dockerignore** → exclude junk/secrets
* **requirements.txt** → runtime Python deps
* **requirements-dev.txt** → dev tooling
* **main.py** → minimal entrypoint/test script
* **claude\_chat.py** → reusable Claude helper functions
* **.env** → API keys (local only, safe from Git/Docker builds)

---

### 🛠️ Build & Run

```bash
# Build runtime image
docker build -t myproject:latest .

# Build dev image (with Jupyter, pytest, etc.)
docker build -t myproject:dev --build-arg INSTALL_DEV=true .

# Run runtime app with secrets
docker run --rm --env-file .env myproject:latest

# Run with interactive shell
docker run -it myproject:latest bash

# Run Jupyter in dev mode
docker run --rm -it -p 8888:8888 myproject:dev \
    jupyter lab --ip=0.0.0.0 --allow-root
```

### 🧹 Cleanup (optional)

```bash
docker builder prune      # clear build cache
docker system prune -a    # remove all unused containers/images
```




# 🚀 Docker Project Template for Data/ML Projects

## 📂 Folder Structure



In [ ]:
myproject/
├── .dockerignore          # excludes junk/secrets from images
├── Dockerfile             # recipe for building container
├── requirements.txt       # runtime dependencies (Claude, rich, etc.)
├── requirements-dev.txt   # developer dependencies (Jupyter, pytest, etc.)
├── .env                   # API keys (never committed, ignored by Git)
│
├── main.py                # minimal entrypoint (system check + hello Claude)
│
├── claude_chat.py         # Claude chat helper functions (multi-turn chat, pretty printing)
│
└── src/                   # your experiments & agent scripts
    ├── chat_loop.py       # interactive REPL with Claude
    ├── data_pipeline.py   # future data/ML experiments
    └── agent_loop.py      # agent orchestration logic


# Build Container Requirement Docs


## 📄 1. `.dockerignore`

Keeps junk, secrets, and large data out of your images.


In [ ]:
nano .dockerignore # then paste the following to create the doc

In [ ]:
# Python cache
__pycache__/
*.pyc
*.pyo
*.pyd

# Virtual environments
venv/
env/
.venv/

# Jupyter Notebook checkpoints
.ipynb_checkpoints/

# OS files
.DS_Store
Thumbs.db

# Git and version control
.git
.gitignore
.gitattributes

# Logs & debug
*.log
*.out
*.err

# Data & models (better to mount at runtime!)
data/
datasets/
*.csv
*.parquet
*.h5
*.pkl

# Large results
outputs/
results/
checkpoints/

# Secrets
.env
*.secret
*.key


## 📄 2. `requirements.txt`

**Runtime essentials** (only what your app needs to run):

* **anthropic** → Claude API client
* **rich** → pretty terminal output (markdown, code blocks)
* **numpy / pandas / scikit-learn** → ML/data science stack (optional, but great if you’re doing experiments)




In [ ]:
nano requirements.txt

In [ ]:
anthropic>=0.37.0
rich==13.7.1
numpy==1.26.4
pandas==2.2.2
scikit-learn==1.5.1

## 📄 3. `requirements-dev.txt`

**Developer extras** (useful for notebooks, testing, formatting):


In [ ]:
nano requirements-dev.txt

In [ ]:
jupyterlab==4.3.0
ipykernel==6.29.5
pytest==8.3.3
flake8==7.1.1
black==24.8.0

## 📄 4. `Dockerfile`

The full environment recipe:

In [ ]:
nano Dockerfile

In [ ]:
FROM python:3.11-slim

# Prevents pyc files, forces flush of stdout/stderr
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

# System deps
RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    curl \
    git \
    wget \
    ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Workdir
WORKDIR /app

# Copy dependency files first (better cache)
COPY requirements.txt requirements-dev.txt ./

# Install runtime deps
RUN pip install --no-cache-dir -r requirements.txt

# Optional dev deps
ARG INSTALL_DEV=false
RUN if [ "$INSTALL_DEV" = "true" ] ; then pip install --no-cache-dir -r requirements-dev.txt ; fi

# Copy code
COPY . .

# Default run
CMD ["python", "main.py"]



## 📄 5. `main.py`

This should stay **minimal**. Its job is to:

* Confirm your container runs correctly (print Python version, OS, args).
* Do a quick “hello world” test with Claude, using helper functions from `claude_chat.py`.
* Act as the **entrypoint** your Dockerfile runs by default.



In [ ]:
nano main.py

In [ ]:
import sys, platform, os
from claude_chat import chat_with_claude, reset_conversation

def main():
    name = os.getenv("NAME", "world")
    print(f"Hello, {name} 👋")
    print("Python:", platform.python_version())
    print("OS:", platform.platform())
    print("Args:", sys.argv[1:])

    reset_conversation()
    chat_with_claude("Hello Claude, can you confirm you’re working?")

if __name__ == "__main__":
    main()




## 📄 6. `claude_chat.py`

This file holds all the **Claude helper functions** so `main.py` can stay clean.




In [ ]:
nano claude_chat.py

In [ ]:
import os, textwrap
from anthropic import Anthropic
from anthropic._exceptions import APIError
from rich.console import Console
from rich.markdown import Markdown

# API setup
anthropic_key = os.getenv("ANTHROPIC_API_KEY")
if not anthropic_key:
    raise RuntimeError("Missing ANTHROPIC_API_KEY environment variable.")

client = Anthropic(api_key=anthropic_key)
MODEL_NAME = "claude-3-5-sonnet-20240620"

# Conversation history
conversation = []

# Pretty printing setup
console = Console()

def smart_print_markdown(output: str, width: int = 100):
    """Wrap plain text, preserve fenced code blocks."""
    in_code = False
    para_buf = []

    def flush_paragraph():
        if para_buf:
            text = " ".join(para_buf)
            print(textwrap.fill(text, width=width, replace_whitespace=False))
            print()
            para_buf.clear()

    for line in output.splitlines():
        fence = line.strip().startswith("```")
        if fence:
            flush_paragraph()
            print(line)
            in_code = not in_code
            continue

        if in_code:
            print(line)
        else:
            if line.strip() == "":
                flush_paragraph()
            else:
                para_buf.append(line)

    flush_paragraph()

def chat_with_claude(
    prompt: str,
    system: str = "You are a helpful coding assistant.",
    render: str = "markdown",      # 'markdown' | 'wrapped' | 'none'
    return_text: bool = False,
    wrap_width: int = 100,
) -> str | None:
    """Send a prompt with conversation memory."""
    if not anthropic_key:
        raise RuntimeError("Missing ANTHROPIC_API_KEY.")

    conversation.append({"role": "user", "content": prompt})

    try:
        msg = client.messages.create(
            model=MODEL_NAME,
            max_tokens=1000,
            temperature=0.2,
            system=system,
            messages=conversation,
        )
        parts = [b.text for b in msg.content if getattr(b, "type", None) == "text"]
        output = "\n\n".join(parts).strip() or "(No text)"

        if render == "markdown":
            console.print(Markdown(output))
        elif render == "wrapped":
            smart_print_markdown(output, width=wrap_width)

        conversation.append({"role": "assistant", "content": output})
        return output if return_text else None

    except APIError as e:
        print("Anthropic API error:", e)
        raise

def reset_conversation():
    """Clear conversation history."""
    conversation.clear()

def last_reply() -> str | None:
    """Get last assistant message."""
    for m in reversed(conversation):
        if m["role"] == "assistant":
            return m["content"]
    return None


## 🔐 7. `.env`

This file stores your **secret environment variables** (like API keys).

In [ ]:
nano.env

In [ ]:
ANTHROPIC_API_KEY=your_api_key_here

## 🛠️ 8: Building Docker Images

In [ ]:
# Build the runtime image
docker build -t myproject:latest .

# Build the dev image (includes Jupyter, pytest, etc.)
docker build -t myproject:dev --build-arg INSTALL_DEV=true .




## Confirm Container Creation
---

### 🔍 Step 1: List all images

```bash
docker images
myproject:latest → your runtime image (leaner).
myproject:dev → your dev image (larger, has Jupyter, pytest, etc.).
```
---

## 🔍 Step 2: Run the runtime image

Test if `main.py` works with your `.env`:

```bash
docker run --rm --env-file .env myproject:latest
```

## 🔍 Step 3: Run the dev image (optional test)

Start Jupyter from inside the dev image:

```bash
docker run --rm -it -p 8888:8888 myproject:dev jupyter lab --ip=0.0.0.0 --allow-root
```




# 💻 Basic Terminal Commands Cheat Sheet

### 📂 File & Folder Navigation

```bash
pwd               # print working directory
ls                # list files
ls -a             # list all files (including hidden)
cd foldername     # change directory
cd ..             # go up one level
```

### 📁 Create / Move / Delete

```bash
mkdir myfolder                # make a new folder
touch file.txt                # create empty file
mv oldname.txt newname.txt    # rename/move file
cp file.txt copy.txt          # copy file
rm file.txt                   # delete file
rm -r myfolder                # delete folder (recursive)
```

### 📄 Viewing Files

```bash
cat file.txt        # show entire file
less file.txt       # scroll through file (press q to quit)
head file.txt       # show first 10 lines
tail file.txt       # show last 10 lines
```

### 📝 Editing Files

```bash
nano file.txt       # edit file in nano (Ctrl+O save, Ctrl+X exit)
```

### ⚙️ System Info (inside container)

```bash
python --version    # check Python version
env                 # list environment variables
which python        # find where python is installed
```


